In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train=pd.read_csv('/kaggle/input/2021mlfinalexamp1/train.csv')
test=pd.read_csv('/kaggle/input/2021mlfinalexamp1/test.csv')

In [3]:
train.head(3)

In [4]:
test.head(3)

In [5]:
train.info()

In [6]:
test.info()

In [7]:
train.isnull().sum()

In [8]:
train['Hour'] = pd.to_numeric(train['Hour'],errors='coerce').astype(float)

In [9]:
train['Minute'] = pd.to_numeric(train['Minute'],errors='coerce').astype(float)
train['DHI'] = pd.to_numeric(train['DHI'],errors='coerce').astype(float)
train['DNI'] = pd.to_numeric(train['DNI'],errors='coerce').astype(float)

In [10]:
train['RH'] = pd.to_numeric(train['RH'],errors='coerce').astype(float)

In [11]:
train.head(3)

In [12]:
train=train.iloc[:,1:9]
test=test.iloc[:,1:9]
train.head(3)

In [13]:
train['WS'] = pd.to_numeric(train['WS'],errors='coerce').astype(float)

In [14]:
train.info(),test.info()

In [15]:
test['Minute'] = pd.to_numeric(test['Minute'],errors='coerce').astype(float)
test['DHI'] = pd.to_numeric(test['DHI'],errors='coerce').astype(float)
test['DNI'] = pd.to_numeric(test['DNI'],errors='coerce').astype(float)
test['Hour'] = pd.to_numeric(test['Hour'],errors='coerce').astype(float)
test['T'] = pd.to_numeric(test['T'],errors='coerce').astype(float)

In [16]:
test['T']=test['T'].astype(int)


In [17]:
train.info(),test.info()

In [18]:
test

In [19]:
train=train.fillna(train.mean())	# 각 열의 평균 값으로 결측치 채우기

In [20]:
train.isna().sum()

In [21]:
test=test.fillna(test.mean())

In [22]:
test.isna().sum()

In [23]:
train.info(), test.info()

In [24]:
y=train['TARGET']
train.drop('TARGET', axis=1, inplace=True)

In [25]:
X=train

In [26]:
X

In [27]:
y

In [28]:
X_train=X
y_train=y


In [29]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from IPython.display import display
import matplotlib.pyplot as plt

In [30]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [32]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

In [33]:
model_gb = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber')

In [34]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [35]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [36]:
model_rf = RandomForestRegressor(n_estimators=3000, 
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10)

In [37]:
score = rmsle_cv(model_gb)
gb_score = score.mean()
print("GradientBoostingRegressor score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [39]:
score = rmsle_cv(model_lgb)
lgb_score = score.mean()
print("LGBMRegressor score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [40]:
score = rmsle_cv(model_xgb)
xgb_score = score.mean()
print("XGBRegressor score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [41]:
score = rmsle_cv(model_rf)
print("RandomForestRegressor score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [44]:
model_xgb.fit(X_train,y_train)

In [45]:
pred=model_xgb.predict(test)

In [50]:
model_gb.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)
model_lgb.fit(X_train, y_train)

In [59]:
X_train.shape

In [61]:
y_train.shape

In [64]:
acc=model_xgb.score(X_train,y_train)

In [65]:
acc

In [51]:
pred_gb = model_gb.predict(test)
pred_xgb = model_xgb.predict(test)
pred_lgb = model_lgb.predict(test)

In [52]:
total_weight = (1. / gb_score) + (1. / xgb_score) + (1. / lgb_score)
pred = (pred_gb * (1. / gb_score) + pred_xgb * (1. / xgb_score) +  pred_lgb * (1. / lgb_score)) / total_weight
total_weight

In [53]:
submission=pd.read_csv('/kaggle/input/2021mlfinalexamp1/sample_submit.csv')

In [54]:
submission['TARGET']=pred

In [55]:
submission.to_csv("submission.csv", mode='w', header=True, index = False)